# Ideal and Actual tables

In [ ]:
# import libraries and data

import pandas as pd
import numpy as np
from os import path
from functools import reduce
from pprint import pprint
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from itertools import combinations

file_dir = "data/consolidated/"
file_dir_old = "data/consolidated/backup/old"

# load the csvs into data frames
df_part_1 = pd.read_csv(path.join(file_dir, "Part 1 - About.csv"))
df_part_3a = pd.read_csv(path.join(file_dir, "Part 3 - Reporting companies' list.csv"))
df_part_3b = pd.read_csv(path.join(file_dir, "Part 3 - Reporting government entities list.csv"))
df_part_3c = pd.read_csv(path.join(file_dir, "Part 3 - Reporting projects' list.csv"))
df_part_4 = pd.read_csv(path.join(file_dir, "Part 4 - Government revenues.csv"))
df_part_5 = pd.read_csv(path.join(file_dir, "Part 5 - Company data.csv"))
# df_part_5 = pd.read_csv(path.join(file_dir, "Part 5 - Company data.csv"), low_memory=False)

df_list = [df_part_1, df_part_3a, df_part_3b, df_part_3c, df_part_4, df_part_5]
df_dict = {"Part 1 - About.csv": df_part_1,
           "Part 3 - Reporting companies' list.csv": df_part_3a,
           "Part 3 - Reporting government entities list.csv": df_part_3b,
           "Part 3 - Reporting projects' list.csv": df_part_3c,
           "Part 4 - Government revenues.csv": df_part_4,
           "Part 5 - Company data.csv": df_part_5
          }

# OPTIONAL COLUMNS
part_3a_opt = ["Stock exchange listing or company website", 
               "Audited financial statement (or balance sheet, cash flows, profit/loss statement if unavailable)"]
part_3b_opt = ["ID number (if applicable)"]
part_5_opt = ["In-kind volume (if applicable)", "Unit (if applicable)", "Comments"]

# only include fields that are non-optional
df_part_1_non_opt = df_part_1.copy()
df_part_3a_non_opt = df_part_3a.copy().drop(columns=part_3a_opt)               
df_part_3b_non_opt = df_part_3b.copy().drop(columns=part_3b_opt)
df_part_3c_non_opt = df_part_3c.copy()
df_part_4_non_opt = df_part_4.copy()
df_part_5_non_opt = df_part_5.copy().drop(columns=part_5_opt)

df_list_non_opt = [df_part_1_non_opt, df_part_3a_non_opt, df_part_3b_non_opt, df_part_3c_non_opt, df_part_4_non_opt, df_part_5_non_opt]
df_dict_non_opt = {"Part 1 - About.csv": df_part_1_non_opt,
           "Part 3 - Reporting companies' list.csv": df_part_3a_non_opt,
           "Part 3 - Reporting government entities list.csv": df_part_3b_non_opt,
           "Part 3 - Reporting projects' list.csv": df_part_3c_non_opt,
           "Part 4 - Government revenues.csv": df_part_4_non_opt,
           "Part 5 - Company data.csv": df_part_5_non_opt
          }

In [ ]:
def compare_tables_drop_duplicates(df1, df2, common_columns_df1, common_columns_df2):
    '''
    Compare two tables based on specified columns and drop duplicates.

    Parameters:
    - df1 (pandas.DataFrame): The first DataFrame.
    - df2 (pandas.DataFrame): The second DataFrame.
    - common_columns_df1 (list): Columns used in df1 to find common rows.
    - common_columns_df2 (list): Columns used in df2 to find common rows.

    Returns:
    - common_rows (pandas.DataFrame): Rows common to both DataFrames with duplicates dropped.
    - unique_rows_df1 (pandas.DataFrame): Rows unique to df1 with duplicates dropped.
    - unique_rows_df2 (pandas.DataFrame): Rows unique to df2 with duplicates dropped.

    Example:
    >>> df1 = pd.DataFrame({'Company': ['A', 'B', 'C'], 'Project name': ['P1', 'P2', 'P3'], 'Country': ['X', 'Y', 'Z'], 'Year': [2020, 2021, 2022]})
    >>> df2 = pd.DataFrame({'Full company name': ['A Corp', 'B Corp', 'D Corp'], 'Company type': ['Type1', 'Type2', 'Type3'], 'Company ID number': [101, 102, 103], 'Country': ['X', 'Y', 'Z'], 'Year': [2020, 2021, 2023]})
    >>> common_cols_df1 = ['Country', 'Year']
    >>> common_cols_df2 = ['Country', 'Year']
    >>> common, unique_df1, unique_df2 = compare_tables_drop_duplicates(df1, df2, common_cols_df1, common_cols_df2)
    >>> print(common)
      Country  Year
    0       X  2020
    1       Y  2021
    >>> print(unique_df1)
      Company Project name
    2       C           P3
    >>> print(unique_df2)
      Full company name Company type  Company ID number
    2            D Corp       Type3                103
    '''

    # Find common rows
    common_rows = pd.merge(df1, df2, left_on=common_columns_df1, right_on=common_columns_df2, how='inner')

    # Drop duplicates in common rows
    common_rows = common_rows.drop_duplicates(subset=common_columns_df1)

    # Drop duplicates in unique rows in df1
    unique_rows_df1 = df1[~df1.set_index(common_columns_df1).index.isin(common_rows.set_index(common_columns_df1).index)]
    unique_rows_df1 = unique_rows_df1.drop_duplicates(subset=common_columns_df1)

    # Drop duplicates in unique rows in df2
    unique_rows_df2 = df2[~df2.set_index(common_columns_df2).index.isin(common_rows.set_index(common_columns_df2).index)]
    unique_rows_df2 = unique_rows_df2.drop_duplicates(subset=common_columns_df2)

    return {"in table 1 but not in table 2": unique_rows_df1, 
            "in table 2 but not in table 1": unique_rows_df2,
            "in both tables": common_rows}


## Part 3a - Reporting companies and Part 5 - Company data

Logic
- Both Part 3a and Part 5 should contain the same companies
- Compute how many rows 
- Create a complete Part 5 by adding the missing companies from Part 3a
- Get the ideal Part 3a (list of companies) from the complete Part 5

Dataframes
- companies_3a_actual = list of companies in part 3a
- companies_5_actual = list of companies in part 5
- companies_3a_missing = list of companies in part 5 but not in part 3a
- companies_5_missing = list of companies in part 3a but not in part 5
- companies_5_actual_complete = companies_5_actual + companies_5_missing
- companies_3a_ideal = list of companies in companies_5_actual_complete

Outcomes
- 1 company is NaN

In [ ]:
df_part_5_allcaps = df_part_5.copy()
df_part_5_allcaps["Company"] = df_part_5_allcaps["Company"].str.upper()
df_part_5_allcaps["Government entity"] = df_part_5_allcaps["Government entity"].str.upper()
df_part_5_allcaps["Project name"] = df_part_5_allcaps["Project name"].str.upper()
df_part_3a_allcaps = df_part_3a.copy()
df_part_3a_allcaps["Full company name"] = df_part_3a_allcaps["Full company name"].str.upper()

common_columns_3a5 = ["Full company name", "Country", "Year"]
common_columns_53a = ["Company", "Country", "Year"]


compare_3a5_allcaps = compare_tables_drop_duplicates(df_part_3a_allcaps, df_part_5_allcaps, common_columns_3a5, common_columns_53a)

print("Duplicate rows removed")
for key, data in compare_3a5_allcaps.items():
    print(f'{key}: {data.shape[0]} rows')

companies_3a_missing_unformatted = compare_3a5_allcaps["in table 2 but not in table 1"]
companies_5_missing_unformatted = compare_3a5_allcaps["in table 1 but not in table 2"]

# display(df_part_5_allcaps)
# display(companies_5_missing_unformatted)

c2k = ["Full company name", "Payments to Governments Report", "Country", "ISO Code", "Year", "Start Date", "End Date"]
companies_5_missing = companies_5_missing_unformatted[c2k].copy()
companies_5_missing = companies_5_missing.rename(columns={"Full company name": "Company", "Payments to Governments Report": "Revenue value"})

companies_5_actual_complete = pd.concat([df_part_5_allcaps, companies_5_missing], ignore_index=True)

print(f"# part 5 rows: {df_part_5_allcaps.shape[0]} \n# missing companies in part 5: {companies_5_missing.shape[0]} \n# of ideal rows in part 5: {df_part_5_allcaps.shape[0] + companies_5_missing.shape[0]} \n# of rows in updated part 5: {companies_5_actual_complete.shape[0]}")

# display(companies_5_missing)
# display(companies_5_actual_complete)

companies_5_actual_complete["Revenue value"] = pd.to_numeric(companies_5_actual_complete["Revenue value"], errors="coerce")
companies_5_actual_complete.to_csv("data/outputs/companies_5_actual_complete.csv", index=False)
pivot_table = companies_5_actual_complete.pivot_table(index=["Company", "Country", "Year", "ISO Code", "Start Date", "End Date"], aggfunc={"Revenue value": "sum"})

display(pivot_table)

# Group the data by Country and Year
grouped_data = companies_5_actual_complete.groupby(['Country', 'Year'])

pivot_tables = []

# Iterate over each group and create a pivot table based on Company
for (country, year), group_df in grouped_data:
    pivot_table = group_df.pivot_table(index='Company', aggfunc={"Revenue value": "sum"}) 
    pivot_table['Country'] = country
    pivot_table['Year'] = year
    pivot_tables.append(pivot_table)

result_df = pd.concat(pivot_tables).reset_index()

result_df = result_df.rename(columns={"Company": "Full company name", "Revenue value": "Payments to Governments Report"})

result_df.to_csv("data/outputs/companies_3a_ideal.csv", index=False)

companies_3a_ideal = result_df.copy()

display(companies_3a_ideal)

companies_5_actual_complete.to_csv("data/outputs/for_cleaning/companies_5_actual_complete.csv", index=False)

In [ ]:
compare_3aideal_5comp_allcaps = compare_tables_drop_duplicates(companies_3a_ideal, companies_5_actual_complete, common_columns_3a5, common_columns_53a)

print("Duplicate rows removed")
for key, data in compare_3aideal_5comp_allcaps.items():
    print(f'{key}: {data.shape[0]} rows')

compare_3aideal_5comp_allcaps["in table 2 but not in table 1"]

In [ ]:
# df_part_3a_allcaps["Full company name"].isna().sum()
df_part_5_allcaps[df_part_5_allcaps["Company"].isnull()]

## Part 3b - Reporting govt agencies and Part 4 - Government revenues

Logic
- Both Part 3b and Part 4 should have the sme government agencies
- Create a complete Part 4 by adding the missing companies from Part 3b
- Get the ideal Part 3b (list of companies) from the complete Part 4

Dataframes
- agencies_3b_actual = list of agencies in part 3b
- agencies_4_actual = list of agencies in part 4
- agencies_3b_missing = list of agencies in part 4 but not in part 3b
- agencies_4_missing = list of agencies in part 3b but not in part 4
- agencies_4_actual_complete = agencies_4_actual + agencies_4_missing
- agencies_3b_ideal = list of agencies in agencies_5_actual_complete

In [ ]:
df_part_4_allcaps = df_part_4.copy()
df_part_4_allcaps["Government entity"] = df_part_4_allcaps["Government entity"].str.upper()
df_part_3b_allcaps = df_part_3b.copy()
df_part_3b_allcaps["Full name of agency"] = df_part_3b_allcaps["Full name of agency"].str.upper()

common_columns_3b4 = ["Full name of agency", "Country", "Year"]
common_columns_43b = ["Government entity", "Country", "Year"]


compare_3b4_allcaps = compare_tables_drop_duplicates(df_part_3b_allcaps, df_part_4_allcaps, common_columns_3b4, common_columns_43b)

print("Duplicate rows removed")
for key, data in compare_3b4_allcaps.items():
    print(f'{key}: {data.shape[0]} rows')

agencies_3b_missing_unformatted = compare_3b4_allcaps["in table 2 but not in table 1"]
agencies_4_missing_unformatted = compare_3b4_allcaps["in table 1 but not in table 2"]

# display(agencies_3b_missing_unformatted)
display(agencies_4_missing_unformatted)

c2ka = ["Full name of agency", "Total reported", "Country", "ISO Code", "Year", "Start Date", "End Date"]
agencies_4_missing = agencies_4_missing_unformatted[c2ka].copy()
agencies_4_missing = agencies_4_missing.rename(columns={"Full name of agency": "Government entity", "Total reported": "Revenue value"})

agencies_4_actual_complete = pd.concat([df_part_4_allcaps, agencies_4_missing], ignore_index=True)

# print(f"# part 5 rows: {df_part_5_allcaps.shape[0]} \n# missing companies in part 5: {companies_5_missing.shape[0]} \n# of ideal rows in part 5: {df_part_5_allcaps.shape[0] + companies_5_missing.shape[0]} \n# of rows in updated part 5: {companies_5_actual_complete.shape[0]}")

# display(df_part_4_allcaps)
# display(agencies_4_actual_complete)

# # display(companies_5_missing)
# display(agencies_4_missing.sort_values(by="Revenue value"))
# display(agencies_4_actual_complete)

agencies_4_actual_complete["Revenue value"] = pd.to_numeric(agencies_4_actual_complete["Revenue value"], errors="coerce")
agencies_4_actual_complete.to_csv("data/outputs/agencies_4_actual_complete.csv", index=False)
# companies_5_actual_complete.to_csv("data/outputs/companies_5_actual_complete.csv", index=False)
agencies_pivot_table = agencies_4_actual_complete.pivot_table(index=["Government entity", "Country", "Year", "ISO Code", "Start Date", "End Date"], aggfunc={"Revenue value": "sum"})

# display(agencies_pivot_table)

# Group the data by Country and Year
agencies_grouped_data = agencies_4_actual_complete.groupby(['Country', 'Year'])

agencies_pivot_tables = []

# Iterate over each group and create a pivot table based on Company
for (country, year), agencies_group_df in agencies_grouped_data:
    agencies_pivot_table = agencies_group_df.pivot_table(index='Government entity', aggfunc={"Revenue value": "sum"}) 
    agencies_pivot_table['Country'] = country
    agencies_pivot_table['Year'] = year
    agencies_pivot_tables.append(agencies_pivot_table)

agencies_result_df = pd.concat(agencies_pivot_tables).reset_index()

agencies_result_df = agencies_result_df.rename(columns={"Government entity": "Full name of agency", "Revenue value": "Total reported"})

agencies_result_df.to_csv("data/outputs/agencies_3b_ideal.csv", index=False)

agencies_3b_ideal = agencies_result_df.copy()

display(agencies_3b_ideal)

agencies_4_actual_complete.to_csv("data/outputs/for_cleaning/agencies_4_actual_complete.csv", index=False)

In [ ]:
compare_3bideal_4comp_allcaps = compare_tables_drop_duplicates(agencies_3b_ideal, agencies_4_actual_complete, common_columns_3b4, common_columns_43b)

print("Duplicate rows removed")
for key, data in compare_3bideal_4comp_allcaps.items():
    print(f'{key}: {data.shape[0]} rows')

compare_3bideal_4comp_allcaps["in table 2 but not in table 1"]

## Part 3c - Reporting projects and Part 5 - Company data

- projects_3c_actual = list of projects in part 3c
- projects_5_actual = list of projects in part 5
- projects_3c_missing = list of projects in part 5 but not in part 3a
- projects_5_missing = list of projects in part 3a but not in part 5
- projects_5_actual_complete = projects_5_actual + projects_5_missing
- projects_3c_ideal = list of projects in projects_5_actual_complete

In [ ]:
# df_part_5_allcaps = df_part_5.copy()
# df_part_5_allcaps["Company"] = df_part_5_allcaps["Company"].str.upper()
# df_part_5_allcaps["Government entity"] = df_part_5_allcaps["Government entity"].str.upper()
# df_part_5_allcaps["Project name"] = df_part_5_allcaps["Project name"].str.upper()
df_part_3c_allcaps = df_part_3c.copy()
df_part_3c_allcaps["Full project name"] = df_part_3c_allcaps["Full project name"].str.upper()
df_part_3c_allcaps["Affiliated companies, start with Operator"] = df_part_3c_allcaps["Affiliated companies, start with Operator"].str.upper()


common_columns_3c5 = ["Full project name", "Country", "Year"]
common_columns_53c = ["Project name", "Country", "Year"]


compare_3c5_allcaps = compare_tables_drop_duplicates(df_part_3c_allcaps, df_part_5_allcaps, common_columns_3c5, common_columns_53c)

print("Duplicate rows removed")
for key, data in compare_3c5_allcaps.items():
    print(f'{key}: {data.shape[0]} rows')

projects_3c_missing_unformatted = compare_3c5_allcaps["in table 2 but not in table 1"]
projects_5_missing_unformatted = compare_3c5_allcaps["in table 1 but not in table 2"]

# display(df_part_5_allcaps)
# display(companies_5_missing_unformatted)

c2kc = ["Full project name", "Country", "ISO Code", "Year", "Start Date", "End Date"]
projects_5_missing = projects_5_missing_unformatted[c2kc].copy()
projects_5_missing = projects_5_missing.rename(columns={"Full project name": "Project name"})

projects_5_actual_complete = pd.concat([df_part_5_allcaps, projects_5_missing], ignore_index=True)

# print(f"# part 5 rows: {df_part_5_allcaps.shape[0]} \n# missing companies in part 5: {companies_5_missing.shape[0]} \n# of ideal rows in part 5: {df_part_5_allcaps.shape[0] + companies_5_missing.shape[0]} \n# of rows in updated part 5: {companies_5_actual_complete.shape[0]}")

# # display(companies_5_missing)
# # display(companies_5_actual_complete)

projects_5_actual_complete["Revenue value"] = pd.to_numeric(projects_5_actual_complete["Revenue value"], errors="coerce")
projects_5_actual_complete.to_csv("data/outputs/projects_5_actual_complete.csv", index=False)
# pivot_table = companies_5_actual_complete.pivot_table(index=["Company", "Country", "Year", "ISO Code", "Start Date", "End Date"], aggfunc={"Revenue value": "sum"})

# display(pivot_table)

# Group the data by Country and Year
projects_grouped_data = projects_5_actual_complete.groupby(['Country', 'Year'])

projects_pivot_tables = []

# Iterate over each group and create a pivot table based on Company
for (country, year), group_df in projects_grouped_data:
    projects_pivot_table = group_df.pivot_table(index='Project name', aggfunc={"Revenue value": "sum"}) 
    projects_pivot_table['Country'] = country
    projects_pivot_table['Year'] = year
    projects_pivot_tables.append(projects_pivot_table)

projects_result_df = pd.concat(projects_pivot_tables).reset_index()

projects_result_df = projects_result_df.rename(columns={"Project name": "Full project name"})

projects_result_df.to_csv("data/outputs/projects_3c_ideal.csv", index=False)

display(projects_result_df)

projects_5_actual_complete.to_csv("data/outputs/for_cleaning/projects_5_actual_complete_v1_simple.csv", index=False)

In [ ]:
# display(projects_3c_missing_unformatted)
display(projects_5_missing_unformatted)

In [ ]:
projects_5_missing_flat = projects_5_missing_unformatted.copy()
display(projects_5_missing_flat)
projects_5_missing_flat.to_csv("data/outputs/projects_5_missing_unformatted.csv", index=False)

projects_5_missing_flat['Affiliated companies, start with Operator'] = projects_5_missing_flat['Affiliated companies, start with Operator'].str.replace(', ', '/').str.replace(',', '/').str.split('/')
display(projects_5_missing_flat)

projects_5_missing_flat_2 = projects_5_missing_flat.explode('Affiliated companies, start with Operator')
projects_5_missing_flat_2['Affiliated companies, start with Operator'] = projects_5_missing_flat_2['Affiliated companies, start with Operator'].str.strip()
display(projects_5_missing_flat_2)
projects_5_missing_flat_2.to_csv("data/outputs/projects_3c_missing_flattened.csv", index=False)
projects_5_missing_flat_2.to_csv("data/outputs/for_cleaning/projects_3c_missing_flattened.csv", index=False)